# Installing Libraries


In [27]:
!pip install langchain openai tiktoken rapidocr-onnxruntime

## Fetch the OpenAi API Key

In [28]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Setting Environment Variable

In [30]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Rag Constist of :
1. Data Ingestion
2. Data Retrival
3. Data Generation

### Data Ingestion

https://en.wikipedia.org/wiki/State_of_the_Union#:~:text=Though%20the%20language%20of%20the,as%20late%20as%20March%207

In [31]:
!pip install -U langchain-community

In [32]:
import requests
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

In [7]:
url = "https://en.wikipedia.org/wiki/State_of_the_Union#:~:text=Though%20the%20language%20of%20the,as%20late%20as%20March%207"

In [8]:
response = requests.get(url)

In [9]:
response

<Response [200]>

In [10]:
response.text

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>State of the Union - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1

In [11]:
raw_Data = response.text

In [12]:
raw_Data

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>State of the Union - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1

In [33]:
with open("state_of_the union.txt", "w") as f:
  f.write(raw_Data)

In [35]:
loader = TextLoader("/content/state_of_the union.txt")

In [36]:
document = loader.load()

In [37]:
document[0].page_content

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with 

## Chunking of the Data

In [38]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [39]:
text_spliter = RecursiveCharacterTextSplitter(chunk_size = 550, chunk_overlap = 75)

In [40]:
text_chunks = text_spliter.split_documents(document)

In [41]:
text_chunks[0]

Document(metadata={'source': '/content/state_of_the union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny.')

In [42]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [43]:
embeddings = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)

In [46]:
pip install faiss-cpu


In [47]:
vectorstore = FAISS.from_documents(text_chunks,embeddings)

In [56]:
retriver = vectorstore.as_retriever()

In [49]:
from langchain.prompts import ChatPromptTemplate

In [50]:
template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [51]:
prompt=ChatPromptTemplate.from_template(template)

In [52]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [53]:
output_parser=StrOutputParser()

In [54]:
llm_model=ChatOpenAI(openai_api_key=OPENAI_API_KEY,model_name="gpt-3.5-turbo")

<ipython-input-54-394d3c7b0376>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_model=ChatOpenAI(openai_api_key=OPENAI_API_KEY,model_name="gpt-3.5-turbo")


In [57]:
rag_chain = (
    {"context": retriver,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [58]:
rag_chain.invoke("How is the United States supporting Ukraine economically and militarily?")

"The United States is supporting Ukraine economically by giving more than $1 billion in direct assistance to aid the Ukrainian people and ease their suffering. The US is inflicting economic pain on Russia and supporting Ukraine by enforcing powerful economic sanctions, cutting off Russia's largest banks from the international financial system, and preventing Russia's central bank from defending the Russian Ruble. American ground forces, air squadrons, and ship deployments have been mobilized to protect NATO countries, including Poland, Romania, Latvia, Lithuania, and Estonia. The US and its Allies will defend every inch of territory of NATO countries with the full force of collective power. The US is not engaged in conflict with Russian forces in Ukraine but is prepared to defend NATO Allies in the event of further aggression from Putin. The Ukrainians are fighting back with courage, and the US stands with the Ukrainian people as they defend their country. The US is choking off Russia'

In [59]:
rag_chain.invoke("What action is the U.S. taking to address rising gas prices?")

"The U.S. is releasing 60 Million barrels of oil from reserves around the world, with 30 Million barrels from its Strategic Petroleum Reserve. This action is aimed at addressing rising gas prices. Additionally, the U.S. is taking robust action to ensure that sanctions target Russia's economy. The government is committed to protecting American businesses and consumers from the impact of these actions. Lowering costs, increasing production in America, and reducing reliance on foreign supply chains are part of the strategy to combat inflation and rising gas prices. The plan includes cutting the cost of prescription drugs, such as insulin, to ease financial burdens on Americans. This comprehensive approach is supported by top economists, business leaders, and a majority of Americans. Overall, the U.S. government is focused on mitigating the impact of rising gas prices through strategic actions and policies."

In [61]:
rag_chain.invoke("how many colours in rainbow")

"I don't know the answer to that question as the retrieved context does not provide information on the number of colors in a rainbow."

In [60]:
rag_chain.invoke("What is deep learning and what are the algorithms of it?")

'Deep learning is a subset of machine learning that uses artificial neural networks to model and solve complex problems. Some common algorithms used in deep learning include Convolutional Neural Networks (CNNs), Recurrent Neural Networks (RNNs), Long Short-Term Memory (LSTM), and Generative Adversarial Networks (GANs). These algorithms are used for tasks such as image recognition, natural language processing, and speech recognition. Deep learning has shown great success in various domains, including healthcare, finance, and autonomous driving. The algorithms of deep learning are designed to learn patterns and features from large amounts of data to make accurate predictions and decisions. They require significant computational resources and large datasets to train effectively.'